In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
import pandas as pd
import os
import numpy as np
df_1 = pd.read_csv('docs/May_anhNA.csv')
df_2 = pd.read_csv('docs/may_cty.csv')

In [ ]:
df = pd.read_csv('docs/List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            df['volume'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            df['dividend'][id] = 'Done'
        if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
            df['table'][id] = 'Done'
df.to_csv('docs/List_company_23052023 - Listing.csv', index = False)

In [3]:
df = pd.read_csv('docs\may_anhNA.csv')
# for id in df.index:
#     symbol = df['Symbol'][id]
#     if os.path.exists(fr'Data\{symbol}\docs\check.csv'):
#         if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
#             df['volume'][id] = 'Done'
#         if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
#             df['dividend'][id] = 'Done'
#         if os.path.exists(fr'Data\{symbol}\docs\checklist_get_table.csv'):
#             df['table'][id] = 'Done'
# df.to_csv('docs\may_anhNA.csv', index = False)
df = pd.read_csv('docs\may_anhNA.csv')
df[['volume', 'dividend']] = np.nan
df.to_csv('docs\may_anhNA.csv', index = False)

df = pd.read_csv('docs\may_cty.csv')
df[['volume', 'dividend']] = np.nan
df.to_csv('docs\may_cty.csv', index = False)

In [4]:
df

,Symbol,check,volume,dividend,table
0,1301,Done,NaN,NaN,Done
1,1332,Done,NaN,NaN,Done
2,1333,Done,NaN,NaN,Done
3,1375,Done,NaN,NaN,Done
4,1376,Done,NaN,NaN,Done
...,...,...,...,...,...
5302,8703,Done,NaN,NaN,Done
5303,8709,Done,NaN,NaN,Done
5304,8729,Done,NaN,NaN,Done
5305,8740,Done,NaN,NaN,Done


In [ ]:
df_1[df_1['check'] == 'Done']

In [3]:
lst_vol = list(df_1[df_1['volume'] == 'Done']['Symbol']) + list(df_2[df_2['volume'] == 'Done']['Symbol'])
len(set(lst_vol))

4122

In [10]:
lst_table = list(df_1[df_1['table'] == 'Done']['Symbol']) + list(df_2[df_2['table'] == 'Done']['Symbol'])
len(set(lst_table))

3615

In [11]:
lst_table = list(df_1[df_1['dividend'] == 'Done']['Symbol']) + list(df_2[df_2['dividend'] == 'Done']['Symbol'])
len(set(lst_table))

3611

In [2]:
from src.ocrPdf import ocrPDF
ocrPDF(r'tests\Data\1332\PDF\2016_Q2_決算短信(2016_11_4).pdf', 'tests/a.pdf')
from src.volume import GetVolume

gv = GetVolume()
gv.findData(path = 'tests/a.pdf')

# move file

In [2]:
%cd ..
import pandas as pd
import numpy as np
import os

a:\Phong\Crawl_buffett


In [3]:
def sortDate(df):
    df['Time'] = pd.to_datetime(df['Time'], format='%Y_%m_%d', errors='coerce')
    df.sort_values(by=['Time'], inplace=True, na_position = 'first')
    df['Time'] = df['Time'].dt.strftime('%d/%m/%Y')
    return df

def getVolume(symbol, PATH = 'Done/volume/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\volume.csv')
    df['volume'] = df['vol1'] - df['vol2']
    df = df[['date', 'volume']]
    df.dropna(inplace=True)
    df.rename(columns={'date': 'Time', 'volume': 'Volume'}, inplace=True)
    df = sortDate(df)
    df.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df

def getDividend(symbol, PATH = 'Done/dividend/'):
    df = pd.read_csv(fr'Data\{symbol}\docs\dividend.csv')
    df_done = pd.DataFrame(columns = ['Time', 'Stock', 'Money', 'Time2'])
    df_done['Time'] = df[['time_split_Q1', 'time_split_Q2', 'time_split_Q3', 'time_split_Q4']].stack(dropna=False).reset_index(drop=True)
    df_done['Time'] = df_done['Time'].str.replace("['", '').str.replace("']", '')
    df_done['Money'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    for q in  ['Q1', 'Q2', 'Q3', 'Q4']:
        df[q] = df['Year'].astype(str) + '_' + q
    df_done['Time2'] = df[['Q1', 'Q2', 'Q3', 'Q4']].stack(dropna=False).reset_index(drop=True)
    df_done.replace('－', np.nan, inplace=True)
    df_done.replace('―', np.nan, inplace=True)
    df_done.replace('-', np.nan, inplace=True)
    df_done.replace('—', np.nan, inplace=True)
    df_done.replace('一', np.nan, inplace=True)
    df_done.replace('=', np.nan, inplace=True)
    df_done.dropna(inplace=True, thresh=2)
    df_done = sortDate(df_done)
    df_done.to_csv(fr'{PATH}{symbol}.csv', index = False)
    return df_done

symbol = 7628
getVolume(symbol)
getDividend(symbol)

,Time,Stock,Money,Time2
1,NaN,NaN,28.0,2022_Q2
3,NaN,NaN,29.0,2022_Q4
5,NaN,NaN,26.0,2021_Q2
7,NaN,NaN,31.0,2021_Q4
9,NaN,NaN,20.0,2020_Q2
11,NaN,NaN,32.0,2020_Q4
13,NaN,NaN,26.0,2019_Q2
15,NaN,NaN,26.0,2019_Q4


In [4]:
os.makedirs(fr'Done/volume', exist_ok=True)
os.makedirs(fr'Done/dividend', exist_ok=True)

df = pd.read_csv('docs\List_company_23052023 - Listing.csv')
for id in df.index:
    symbol = df['Symbol'][id]
    if df['check'][id] == 'Done':
        if os.path.exists(fr'Data\{symbol}\docs\volume.csv'):
            try:
                getVolume(symbol)
            except Exception as e:
                print(symbol, e)
                break

        if os.path.exists(fr'Data\{symbol}\docs\dividend.csv'):
            try:
                getDividend(symbol)
            except Exception as e:
                print(symbol, e)
                break

7699 Can only use .str accessor with string values!
